# Scikit-Learn IRIS Model using jsonData

 * Wrap a scikit-learn python model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Dependencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip install sklearn
pip install seldon-core
```

## Train locally
 

In [1]:
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn import datasets

def main():
    clf = LogisticRegression()
    p = Pipeline([('clf', clf)])
    print('Training model...')
    p.fit(X, y)
    print('Model trained!')

    filename_p = 'IrisClassifier.sav'
    print('Saving model in %s' % filename_p)
    joblib.dump(p, filename_p)
    print('Model saved!')
    
if __name__ == "__main__":
    print('Loading iris data set...')
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    print('Dataset loaded!')
    main()


Loading iris data set...
Dataset loaded!
Training model...
Model trained!
Saving model in IrisClassifier.sav
Model saved!


## REST test

Wrap model using s2i

In [2]:
!s2i build . seldonio/seldon-core-s2i-python3:0.13 sklearn-iris-jsondata:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


Serve model

In [3]:
!docker run --name "iris_predictor" -d --rm -p 5000:5000 sklearn-iris-jsondata:0.1

c72f210d5669ede4055990e3ea2b4d167a1a5c5b17d0cd3b3837e5bf17fbc4e5


In [4]:
%%bash

#
# Send some random features to model
#

curl -sg http://localhost:5000/predict --data-urlencode 'json=
{
    "jsonData": {
        "some_data": {
            "names": [
                "sepal_length",
                "sepal_width",
                "petal_length",
                "petal_width"
            ],
            "some_ndarray": [
                [
                    7.233,
                    4.652,
                    7.39,
                    0.324
                ]
            ]
        }
    }
}' | jq

{
  "data": {
    "names": [
      "t:0",
      "t:1",
      "t:2"
    ],
    "tensor": {
      "shape": [
        1,
        3
      ],
      "values": [
        0.0011809149535246906,
        0.7867455841528153,
        0.21207350089366006
      ]
    }
  },
  "meta": {}
}


Stop serving model

In [5]:
!docker rm iris_predictor --force

iris_predictor


## Test using Minikube


In [ ]:
!minikube start --memory 4096

## Setup Seldon Core

Follow the instructions to [Setup Cluster](../../seldon_core_setup.ipynb#Setup-Cluster) with [Ambassador Ingress](../../seldon_core_setup.ipynb#Ambassador) and [Install Seldon Core](../../seldon_core_setup.ipynb#Install-Seldon-Core).

## Wrap Model and Test

In [14]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-python3:0.13 sklearn-iris-jsondata:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [15]:
!kubectl create -f sklearn_iris_jsondata_deployment.json

seldondeployment.machinelearning.seldon.io/seldon-deployment-example created


In [16]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-app=sklearn-iris-deployment-sklearn-iris-predictor --no-headers=true|cut -d' ' -f1)

Waiting for deployment "sklearn-iris-deployment-sklearn-iris-predictor-5d3a732" rollout to finish: 0 of 1 updated replicas are available...
deployment "sklearn-iris-deployment-sklearn-iris-predictor-5d3a732" successfully rolled out


### Test by sending prediction calls

In [18]:
%%bash

PAYLOAD=$(
cat <<EOF
{
    "jsonData": {
        "some_data": {
            "names": [
                "sepal_length",
                "sepal_width",
                "petal_length",
                "petal_width"
            ],
            "some_ndarray": [
                [
                    7.233,
                    4.652,
                    7.39,
                    0.324
                ]
            ]
        }
    }
}
EOF
)

NAMESPACE_NAME=default
DEPLOYMENT_NAME=seldon-deployment-example

HOST=$(minikube ip)
PORT=$(kubectl get svc ambassador -o jsonpath='{.spec.ports[0].nodePort}')

set -x
curl -s -H "Content-Type: application/json" \
    -d "${PAYLOAD}" \
    "http://${HOST}:${PORT}/seldon/${NAMESPACE_NAME}/${DEPLOYMENT_NAME}/api/v0.1/predictions" | jq
set +x

{
  "meta": {
    "puid": "6d6jdiqbbtd01qbqdapjmlqc6d",
    "tags": {},
    "routing": {},
    "requestPath": {
      "sklearn-iris-classifier": "sklearn-iris-jsondata:0.1"
    },
    "metrics": []
  },
  "data": {
    "names": [
      "t:0",
      "t:1",
      "t:2"
    ],
    "tensor": {
      "shape": [
        1,
        3
      ],
      "values": [
        0.0011809149535246906,
        0.7867455841528153,
        0.21207350089366006
      ]
    }
  }
}


+ jq
+ curl -s -H 'Content-Type: application/json' -d '{
    "jsonData": {
        "some_data": {
            "names": [
                "sepal_length",
                "sepal_width",
                "petal_length",
                "petal_width"
            ],
            "some_ndarray": [
                [
                    7.233,
                    4.652,
                    7.39,
                    0.324
                ]
            ]
        }
    }
}' http://192.168.39.230:30241/seldon/default/seldon-deployment-example/api/v0.1/predictions
+ set +x


In [ ]:
!minikube delete